In [1]:
import os
os.chdir("/work/pi_jensen_umass_edu/jnainani_umass_edu/CircuitAnalysisSAEs/")
from functools import partial
from typing import Callable, Optional, Sequence, Tuple, Union, overload
import einops
import pandas as pd
import torch
from jaxtyping import Float, Int
from tqdm.auto import tqdm
from typing_extensions import Literal
from transformer_lens.utils import Slice, SliceInput
import sys 
import functools
import re
from collections import defaultdict
from sae_lens.toolkit.pretrained_saes_directory import get_pretrained_saes_directory
import json
import os 
from sae_lens import SAE, HookedSAETransformer
# import error_data
sys.path.append("utils/")
import plot

with open("config.json", 'r') as file:
    config = json.load(file)
    token = config.get('huggingface_token', None)
os.environ["HF_TOKEN"] = token
if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
hf_cache = "/work/pi_jensen_umass_edu/jnainani_umass_edu/mechinterp/huggingface_cache/hub"
os.environ["HF_HOME"] = "/work/pi_jensen_umass_edu/jnainani_umass_edu/mechinterp/huggingface_cache/hub"

model = HookedSAETransformer.from_pretrained("google/gemma-2-9b", device = device, cache_dir = hf_cache)

Device: cuda


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2-9b into HookedTransformer


In [2]:

from transformer_lens.utils import test_prompt

prompt = """def f(nums):
    output = []
    for n in nums:
        output.append((nums.count(n), n))
    output.sort(reverse=True)
    return output
    
assert f([1, 1, 3, 1, 3, 1]) == [("""
test_prompt(prompt, " TypeError", model)



Tokenized prompt: ['<bos>', 'def', ' f', '(', 'nums', '):', '\n', '    ', 'output', ' =', ' []', '\n', '    ', 'for', ' n', ' in', ' nums', ':', '\n', '        ', 'output', '.', 'append', '((', 'nums', '.', 'count', '(', 'n', '),', ' n', '))', '\n', '    ', 'output', '.', 'sort', '(', 'reverse', '=', 'True', ')', '\n', '    ', 'return', ' output', '\n', '    ', '\n', 'assert', ' f', '([', '1', ',', ' ', '1', ',', ' ', '3', ',', ' ', '1', ',', ' ', '3', ',', ' ', '1', '])', ' ==', ' [(']
Tokenized answer: [' TypeError']


Performance on answer token:
Rank: 72056    Logit: -0.02 Prob:  0.00% Token: | TypeError|

Top 0th token. Logit: 29.79 Prob: 66.17% Token: |3|
Top 1th token. Logit: 28.87 Prob: 26.38% Token: |4|
Top 2th token. Logit: 26.79 Prob:  3.28% Token: |2|
Top 3th token. Logit: 26.46 Prob:  2.37% Token: |5|
Top 4th token. Logit: 25.73 Prob:  1.14% Token: |6|
Top 5th token. Logit: 24.81 Prob:  0.46% Token: |1|
Top 6th token. Logit: 23.13 Prob:  0.09% Token: |7|
Top 7th token. Logit: 22.36 Prob:  0.04% Token: |0|
Top 8th token. Logit: 21.93 Prob:  0.03% Token: |8|
Top 9th token. Logit: 21.90 Prob:  0.02% Token: |9|


Ranks of the answer tokens: [(' TypeError', 72056)]

In [3]:
print(model.generate(prompt, max_new_tokens=20))

  0%|          | 0/20 [00:00<?, ?it/s]

def f(nums):
    output = []
    for n in nums:
        output.append((nums.count(n), n))
    output.sort(reverse=True)
    return output
    
assert f([1, 1, 3, 1, 3, 1]) == [(3, 1), (2, 3)]
assert f([1, 2,


# Important features in Benchmarks 


Idea: 
- pick a benchmark (newish) 
- For each forward pass, save the top k latents with 0 attribution patching 
- backward pass wrt - ce loss ? 
- manually or thorugh an LLM pick "key" tokens relevant to the answer
- Average attribution over latents for these tokens over all data points 
- find cool and relevant latents that might potentially explain performance on the benchmark 

Why not? 
- too ambitious 
- very unprincipled 
- bag of tricks might cancel things out 


# Error detection

In [29]:
prompt = """Type "help", "copyright", "credits" or "license" for more information.
>>> my_dict = {"a": 1, "b": 2, "c": 3}
>>> for key in ["a", "b", "d"]:
...     print(my_dict[key])
"""

print(model.generate(prompt, max_new_tokens=20))

  0%|          | 0/20 [00:00<?, ?it/s]

Type "help", "copyright", "credits" or "license" for more information.
my_dict = {"a": 1, "b": 2, "c": 3}
for key in ["a", "b", "d"]:
    print(my_dict[key])
error
error is a built in type
my_dict = {"a": 1, "


In [34]:
prompt = """Type "help", "copyright", "credits" or "license" for more information.
>>> fields = ["name", "age", "city"]
>>> data = {"name": "Alice", "age": 25}
>>> for field in fields:
...     print(f"{field}: {data[field]}")
... 
name: Alice
age: 25
"""
test_prompt(prompt, "Traceback", model, prepend_space_to_answer=False)

Tokenized prompt: ['<bos>', 'Type', ' "', 'help', '",', ' "', 'copyright', '",', ' "', 'credits', '"', ' or', ' "', 'license', '"', ' for', ' more', ' information', '.', '\n', 'fields', ' =', ' ["', 'name', '",', ' "', 'age', '",', ' "', 'city', '"]', '\n', 'data', ' =', ' {"', 'name', '":', ' "', 'Alice', '",', ' "', 'age', '":', ' ', '2', '5', '}', '\n', 'for', ' field', ' in', ' fields', ':', '\n', '    ', 'print', '(', 'f', '"{', 'field', '}:', ' {', 'data', '[', 'field', ']}', '")', '\n\n', 'name', ':', ' Alice', '\n', 'age', ':', ' ', '2', '5', '\n']
Tokenized answer: ['Traceback']


Performance on answer token:
Rank: 1        Logit: 24.64 Prob: 12.55% Token: |Traceback|

Top 0th token. Logit: 25.95 Prob: 46.44% Token: |<eos>|
Top 1th token. Logit: 24.64 Prob: 12.55% Token: |Traceback|
Top 2th token. Logit: 24.03 Prob:  6.78% Token: |city|
Top 3th token. Logit: 23.93 Prob:  6.16% Token: |>>>|
Top 4th token. Logit: 23.35 Prob:  3.44% Token: |```|
Top 5th token. Logit: 23.15 Prob:  2.83% Token: |name|
Top 6th token. Logit: 22.99 Prob:  2.40% Token: |</code>|
Top 7th token. Logit: 22.64 Prob:  1.70% Token: |----------------|
Top 8th token. Logit: 22.04 Prob:  0.93% Token: |'''|
Top 9th token. Logit: 21.92 Prob:  0.82% Token: |Syntax|


Ranks of the answer tokens: [('Traceback', 1)]

In [35]:
print("from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    \"\"\" Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    \"\"\"\n    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n")

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False



In [51]:
prompt = """You are given a Python function and a question on the function. Answer the question with only YES or NO, considering executing the provided code on the given input, even if the function is incorrect or incomplete. Do NOT output any extra information.

```python
from typing import List

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    # Check if in given list of numbers, are any two numbers closer to each other than given threshold.
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2) # Line A
                if distance < threshold: 
                    return True # Line B

    return False
has_close_elements([1.0, 2.0, 3.0], 0.5)
```
Question: Will Line B be executed?
Answer:"""
test_prompt(prompt, "YES", model, prepend_space_to_answer=False)

Tokenized prompt: ['<bos>', 'You', ' are', ' given', ' a', ' Python', ' function', ' and', ' a', ' question', ' on', ' the', ' function', '.', ' Answer', ' the', ' question', ' with', ' only', ' YES', ' or', ' NO', ',', ' considering', ' executing', ' the', ' provided', ' code', ' on', ' the', ' given', ' input', ',', ' even', ' if', ' the', ' function', ' is', ' incorrect', ' or', ' incomplete', '.', ' Do', ' NOT', ' output', ' any', ' extra', ' information', '.', '\n\n', '```', 'python', '\n', 'from', ' typing', ' import', ' List', '\n\n', 'def', ' has', '_', 'close', '_', 'elements', '(', 'numbers', ':', ' List', '[', 'float', '],', ' threshold', ':', ' float', ')', ' ->', ' bool', ':', '\n', '    ', '#', ' Check', ' if', ' in', ' given', ' list', ' of', ' numbers', ',', ' are', ' any', ' two', ' numbers', ' closer', ' to', ' each', ' other', ' than', ' given', ' threshold', '.', '\n', '    ', 'for', ' idx', ',', ' elem', ' in', ' enumerate', '(', 'numbers', '):', '\n', '        ', 

Performance on answer token:
Rank: 26       Logit: 21.68 Prob:  0.20% Token: |YES|

Top 0th token. Logit: 26.84 Prob: 35.32% Token: | YES|
Top 1th token. Logit: 26.21 Prob: 18.89% Token: | NO|
Top 2th token. Logit: 25.57 Prob:  9.92% Token: |
|
Top 3th token. Logit: 24.96 Prob:  5.41% Token: |<eos>|
Top 4th token. Logit: 24.50 Prob:  3.42% Token: | |
Top 5th token. Logit: 24.45 Prob:  3.24% Token: |

|
Top 6th token. Logit: 24.43 Prob:  3.19% Token: | Yes|
Top 7th token. Logit: 24.29 Prob:  2.75% Token: | [|
Top 8th token. Logit: 23.95 Prob:  1.96% Token: | No|
Top 9th token. Logit: 23.73 Prob:  1.58% Token: | `|


Ranks of the answer tokens: [('YES', 26)]

# Code Coverage Prediction

In [46]:
prompt = """
You are given a Python function and a question on the function. Answer the question with only YES or NO, considering executing the provided code on the given input, even if the function is incorrect or incomplete. Do NOT output any extra information.
```python
x = 2
if x > 5:
    print("Greater than 5")  # Line A
else:
    print("Not greater than 5")  # Line B
```
Question: Will Line B be executed?
Answer:"""
test_prompt(prompt, "YES", model, prepend_space_to_answer=False)

Tokenized prompt: ['<bos>', '\n', 'You', ' are', ' given', ' a', ' Python', ' function', ' and', ' a', ' question', ' on', ' the', ' function', '.', ' Answer', ' the', ' question', ' with', ' only', ' YES', ' or', ' NO', ',', ' considering', ' executing', ' the', ' provided', ' code', ' on', ' the', ' given', ' input', ',', ' even', ' if', ' the', ' function', ' is', ' incorrect', ' or', ' incomplete', '.', ' Do', ' NOT', ' output', ' any', ' extra', ' information', '.', '\n', '```', 'python', '\n', 'x', ' =', ' ', '2', '\n', 'if', ' x', ' >', ' ', '5', ':', '\n', '    ', 'print', '("', 'Greater', ' than', ' ', '5', '")', '  ', '#', ' Line', ' A', '\n', 'else', ':', '\n', '    ', 'print', '("', 'Not', ' greater', ' than', ' ', '5', '")', '  ', '#', ' Line', ' B', '\n', '```', '\n', 'Question', ':', ' Will', ' Line', ' B', ' be', ' executed', '?', '\n', 'Answer', ':']
Tokenized answer: ['YES']


Performance on answer token:
Rank: 22       Logit: 21.11 Prob:  0.17% Token: |YES|

Top 0th token. Logit: 26.47 Prob: 34.85% Token: | YES|
Top 1th token. Logit: 26.36 Prob: 31.41% Token: | NO|
Top 2th token. Logit: 25.15 Prob:  9.38% Token: |
|
Top 3th token. Logit: 23.97 Prob:  2.88% Token: | No|
Top 4th token. Logit: 23.82 Prob:  2.48% Token: | Yes|
Top 5th token. Logit: 23.76 Prob:  2.33% Token: | `|
Top 6th token. Logit: 23.63 Prob:  2.04% Token: | |
Top 7th token. Logit: 23.50 Prob:  1.79% Token: | ```|
Top 8th token. Logit: 23.42 Prob:  1.66% Token: |

|
Top 9th token. Logit: 23.42 Prob:  1.65% Token: | [|


Ranks of the answer tokens: [('YES', 22)]

In [49]:
prompt = """```python
def check(x):
    if x > 0:
        print("Positive")  # Line A

check(-1)
```
Question: Will Line A be executed?
Answer:"""
test_prompt(prompt, "YES", model, prepend_space_to_answer=False)

Tokenized prompt: ['<bos>', '```', 'python', '\n', 'def', ' check', '(', 'x', '):', '\n', '    ', 'if', ' x', ' >', ' ', '0', ':', '\n', '        ', 'print', '("', 'Positive', '")', '  ', '#', ' Line', ' A', '\n\n', 'check', '(-', '1', ')', '\n', '```', '\n', 'Question', ':', ' Will', ' Line', ' A', ' be', ' executed', '?', '\n', 'Answer', ':']
Tokenized answer: ['YES']


Performance on answer token:
Rank: 352      Logit: 16.59 Prob:  0.00% Token: |YES|

Top 0th token. Logit: 27.53 Prob: 46.12% Token: | No|
Top 1th token. Logit: 26.08 Prob: 10.84% Token: | Yes|
Top 2th token. Logit: 25.80 Prob:  8.18% Token: | False|
Top 3th token. Logit: 24.77 Prob:  2.91% Token: |
|
Top 4th token. Logit: 24.76 Prob:  2.90% Token: | Line|
Top 5th token. Logit: 24.58 Prob:  2.43% Token: | True|
Top 6th token. Logit: 24.29 Prob:  1.81% Token: | |
Top 7th token. Logit: 24.29 Prob:  1.80% Token: |  |
Top 8th token. Logit: 24.25 Prob:  1.73% Token: | A|
Top 9th token. Logit: 24.24 Prob:  1.71% Token: | NO|


Ranks of the answer tokens: [('YES', 352)]